In [1]:
import pyautogui as pag
import time
import webbrowser
import pandas as pd

In [2]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.layers import concatenate
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
import cv2
import numpy as np 


def combine_4_image(img1, img2, img3, img4):
    img1 = cv2.resize(img1, (112, 112))
    img2 = cv2.resize(img2, (112, 112))
    img3 = cv2.resize(img3, (112, 112))
    img4 = cv2.resize(img4, (112, 112))

    output = np.zeros((225, 225, 3))
    output[0:112, 0:112] = img1
    output[113:225, 0:112] = img2
    output[0:112, 113:225] = img3
    output[113:225, 113:225] = img4
    return output[0:224, 0:224]


def normalize(img):
    return img / 255


def numpy_minmax(x):
    xmin =  np.asarray([-38.03926279, -5.62667684, -27.69314801, -1.5502475])
    xmax = np.asarray([-0.03637159, -0.87553305, 23.91591131, 1.55643957])
    scaler = (x - xmin) / (xmax - xmin)
    scaler = scaler * .6
    return scaler + .2


def build_model(path_to_weights):
    images = Input(shape=(224,224,3),name = 'image_input')
    non_image_df = Input(shape=(19,),name = 'non_image_input')
    inception_v3_model = InceptionV3(weights='imagenet', include_top=False)
    for layer in inception_v3_model.layers:
        layer.trainable=False
    inception_v3_model.summary()

    #Use the generated model 
    inception_v3_model_conv = inception_v3_model(images)
    #forward = GlobalAveragePooling2D()(inception_v3_model_conv)
    cnn = GlobalAveragePooling2D()(inception_v3_model_conv)
    final = concatenate([cnn, non_image_df])    #forward
    final = Dense(1024,activation="relu")(final)
#    final = Dropout(0.5)(final)
    final = Dense(1024,activation="relu")(final)
#    final = Dropout(0.5)(final)
    final = Dense(6, activation='linear')(final)
    model = Model(input=[images, non_image_df], output=final)
#Create model 
    model.load_weights(path_to_weights)
    adam = Adam(lr=0.001, clipvalue=1.5)
    model.compile(loss='mse', optimizer=adam)
    return model


class inference_runner():
    def __init__(self, path_to_weights):
        self.__trained_model = build_model(path_to_weights)

    def run_inference(self, non_img_inputs, forward_last_img,
                      down_last_img, forward_current_img,
                      down_current_img):
        noisy_positions = non_img_inputs[:4]
        scaled_positions = list(numpy_minmax(noisy_positions))
        scaled_positions.extend(non_img_inputs[4:])
        print("scaled_positions: ", scaled_positions)
        scaled_positions = np.asarray(scaled_positions)
        combined_img = combine_4_image(forward_last_img, down_last_img,
                                       forward_current_img,
                                       down_current_img)
        norm_img = normalize(combined_img)
        tensor_img = norm_img.reshape((1, 224, 224, 3))
        tensor_positions = scaled_positions.reshape(1, 19)
        return self.__trained_model.predict([tensor_img, tensor_positions])

# Tests
img_path = "/home/joe/Dev/robot/auto pilot batch 1/{}"
forward_last_img_path = img_path.format("autofwdB1.png")
down_last_img_path = img_path.format("autofwdB1.png")
forward_current_img_path = img_path.format("autofwdB2.png")
down_current_img_path = img_path.format("autofwdB2.png")
forward_last_img = cv2.imread('{}'.format(forward_last_img_path))
down_last_img = cv2.imread('{}'.format(down_last_img_path))
forward_current_img = cv2.imread('{}'.format(forward_current_img_path))
down_current_img = cv2.imread('{}'.format(down_current_img_path))
non_img_inputs = [.5, .5, .5, .5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#model_path = '/home/joe/Dev/robot/models/InceptionV3_fc512_X2_pos-33.hdf5'
model_path = '/home/joe/Dev/robot_direction_control/models/InceptionV3_fc1024_X2_pos83_epoch.hdf5'
position_predictor = inference_runner(model_path)
output = position_predictor.run_inference(non_img_inputs, forward_last_img,
                      down_last_img, forward_current_img,
                      down_current_img)
print(output)

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

/home/joe/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


scaled_positions:  [0.80846838079519601, 0.97370971422441421, 0.52776975649010915, 0.59596794665257358, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[  6.00480413 -23.90739059   0.7720902    0.09028946   0.15408972
   -0.03125475]]


In [3]:
model_path = '/home/joe/Dev/robot_direction_control/models/InceptionV3_fc1024_X2_pos83_epoch.hdf5'
position_predictor = inference_runner(model_path)
screenWidth, screenHeight = pag.size()
pag.moveTo(screenWidth / 2, screenHeight / 2)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_95[0][0]     
___________________________________________________________________________________________

/home/joe/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [4]:
def load_image(filename1, filename2, filename3, filename4):
    img1 = cv2.imread('{}'.format(filename1))
    img2 = cv2.imread('{}'.format(filename2))
    img3 = cv2.imread('{}'.format(filename3))
    img4 = cv2.imread('{}'.format(filename4))
    img1 = cv2.resize(img1, (112, 112))
    img2 = cv2.resize(img2, (112, 112))
    img3 = cv2.resize(img3, (112, 112))
    img4 = cv2.resize(img4, (112, 112))

    output = np.zeros((225, 225, 3))
    output[0:112,0:112] = img1
    output[113:225,0:112] = img2
    output[0:112,113:225] = img3
    output[113:225,113:225] = img4
    return output[0:224,0:224]

def mean_normalize(img):
    return (img - img.mean()) / (img.max() - img.min())

def normalize(img):
    return img / 255

def is_nan(value):
    if value > 0:
        return False
    if value < 0:
        return False
    if value == 0:
        return False
    return True

def convert_y_wasd(input_value):
    if input_value == False:

        return 0
    if input_value == True:
        return 1
    return input_value



In [5]:
import glob
import os
def get_latest_file():
    """Returns the name of the latest (most recent) file 
    of the joined path(s)"""
    list_of_files = glob.glob('/home/joe/Downloads/*.csv') 
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file

def get_last_point():
    filename = get_latest_file()
    print(filename)
    df = pd.read_csv(filename)
    bottom = df.tail(2)
    last_index = df.index[-1]
    forward_current = bottom['filename1'][last_index]
    down_current = bottom['filename2'][last_index]
    forward_last = bottom['filename1'][last_index-1]
    down_last = bottom['filename2'][last_index-1]
    
    position_x = bottom['position.x'][last_index]
    position_y = bottom['position.y'][last_index]
    position_z = bottom['position.z'][last_index]
    rotation_y = bottom['rotation.y'][last_index]
    validationgate_points = convert_y_wasd(bottom['validationgatePoints'][last_index])
    pathmarker1_points = convert_y_wasd(bottom['pathmarker1Points'][last_index])
    buoy1_points = convert_y_wasd(bottom['buoy1Points'][last_index])
    buoy1_backup = convert_y_wasd(bottom['buoy1Backup'][last_index])
    buoy2_points = convert_y_wasd(bottom['buoy2Points'][last_index])
    buoy2_backup = convert_y_wasd(bottom['buoy2Backup'][last_index])
    buoy3_points = convert_y_wasd(bottom['buoy3Points'][last_index])
    pathmarker2_points = convert_y_wasd(bottom['pathmarker2Points'][last_index])
    maneuver_points = convert_y_wasd(bottom['maneuverPoints'][last_index])
    pathmarker3_points = convert_y_wasd(bottom['pathmarker3Points'][last_index])
    pathmarker4_points = convert_y_wasd(bottom['pathmarker4Points'][last_index])
    pathmarker5_points = convert_y_wasd(bottom['pathmarker5Points'][last_index])
    octagon1_points = convert_y_wasd(bottom['octagon1Points'][last_index])
    pathmarker6_points = convert_y_wasd(bottom['pathmarker6Points'][last_index])
    pathmarker7_points = convert_y_wasd(bottom['pathmarker7Points'][last_index])
    octagon2_points = convert_y_wasd(bottom['octagon2Points'][last_index])
    return [forward_current, down_current, forward_last, down_last,
            position_x, position_y, position_z, rotation_y,
            validationgate_points, pathmarker1_points,
            buoy1_points, buoy1_backup, buoy2_points, buoy2_backup, buoy3_points,
            pathmarker2_points, maneuver_points, pathmarker3_points,
            pathmarker4_points, pathmarker5_points, octagon1_points,pathmarker6_points,
            pathmarker7_points, octagon2_points
           ]


In [6]:
def get_out_put():
    last_point = get_last_point()
    img_path = "/home/joe/Downloads/{}"
    forward_last_img_path = img_path.format(str(last_point[0]))
    print(forward_last_img_path)
    down_last_img_path = img_path.format(str(last_point[1]))
    print(down_last_img_path)
    forward_current_img_pathss = img_path.format(str(last_point[2]))
    print(forward_current_img_path)
    down_current_img_path = img_path.format(str(last_point[3]))
    print(down_current_img_path)
    forward_last_img = cv2.imread('{}'.format(forward_last_img_path))
    down_last_img = cv2.imread('{}'.format(down_last_img_path))
    forward_current_img = cv2.imread('{}'.format(forward_current_img_path))
    down_current_img = cv2.imread('{}'.format(down_current_img_path))
    non_img_inputs = last_point[5:]
    #[-4.9278298848, -1.045838346, -25.872343062,1.2358679902,0,0,0,0,0,0,0,0,0,0]
    #model_path = '/home/joe/Dev/robot/models/InceptionV3_fc512_X2_pos-33.hdf5'
    model_path = '/home/joe/Dev/robot_direction_control/models/InceptionV3_fc1024_X2_pos83_epoch.hdf5'
    output = position_predictor.run_inference(non_img_inputs, forward_last_img,
                          down_last_img, forward_current_img,
                          down_current_img)
    return output


Quick test script

In [7]:

"""
A script to illustrate PyKeyboard use
"""

import pykeyboard
import time

k = pykeyboard.PyKeyboard()

def press_and_hold(character, hold_time):
    k.press_key(character)
    time.sleep(hold_time)
    k.release_key(character)



press_and_hold('W', 1)


In [8]:
def clip_value(limit_size, value):
    return value
    if float(value) < -1.0 * float(limit_size):
        return -1.0 * float(limit_size)
    if float(value) > flaot(limit_size) :
        return float(limit_size)
    return float(limit_size)


In [ ]:

webbrowser.open('http://cantren.github.io/')
filename = "/home/joe/Downloads/labels.csv"
screenWidth, screenHeight = pag.size()
time.sleep(.1)
width_offset = 20
pag.moveTo(screenWidth / 2, screenHeight / 2)
time.sleep(20)
press_and_hold('R', .5)
time.sleep(20)
horoz_1 = 0
horoz_2 = 0
horoz_3 = 0
horoz_4 = 0
verticle_1 = 0
verticle_2 = 0
verticle_3 = 0
verticle_4 = 0
multiplier = 3
clip = 70
for i in range(0,501):
    try:
        output = get_out_put().tolist()[0]
        #print(output)
        time.sleep(.2)
        horoz_1 = clip_value(clip, output[0])
        horoz_tot = (horoz_1 + horoz_2 * multiplier + horoz_3 + horoz_4)/4
        verticle_1 = clip_value(clip, output[1])
        verticle_tot = (verticle_1 * multiplier + verticle_2 + verticle_3 + verticle_4)/18
        pag.moveTo((screenWidth / 2) + horoz_tot + width_offset, (screenHeight / 2) + verticle_tot + 55)
        time.sleep(.2)
        #print(output[2])
        if output[2] > .5:
            print("Pressed W")
            press_and_hold('W', .15)
        output = get_out_put().tolist()[0]
        time.sleep(.2)
        horoz_2 = clip_value(clip, output[0])
        horoz_tot = (horoz_1 + horoz_2 * multiplier + horoz_3 + horoz_4)/6
        verticle_2 = clip_value(10, output[1])
        verticle_tot = (verticle_1 + verticle_2 * multiplier + verticle_3 + verticle_4)/18
        pag.moveTo((screenWidth / 2) + horoz_tot + width_offset, (screenHeight / 2) + verticle_tot + 58)
        time.sleep(.2)
        if output[3] > .5:
            print("Pressed A")
            press_and_hold('A', .15)
        output = get_out_put().tolist()[0]
        time.sleep(.2)
        horoz_3 = clip_value(clip, output[0])
        horoz_tot = (horoz_1 + horoz_2 + horoz_3 * multiplier + horoz_4)/6
        verticle_3 = clip_value(clip, output[1])
        verticle_tot = (verticle_1 + verticle_2 + verticle_3 * multiplier + verticle_4)/18
        pag.moveTo((screenWidth / 2) + horoz_tot + width_offset, (screenHeight / 2) + verticle_tot + 58)
        time.sleep(.2)
        if output[3] > .5:
            press_and_hold('S', .15)
            print("Pressed S")
        output = get_out_put().tolist()[0]
        time.sleep(.2)
        horoz_4 = clip_value(clip, output[0])
        horoz_tot = (horoz_1 + horoz_2 + horoz_3 + horoz_4 * multiplier)/6
        verticle_4 = clip_value(clip, output[1])
        verticle_tot = (verticle_1 + verticle_2 + verticle_3 + verticle_4 * multiplier)/18
        pag.moveTo((screenWidth / 2) + horoz_tot + width_offset, (screenHeight / 2) + verticle_tot + 58)
        time.sleep(.2)
        if output[4] > .4:
            press_and_hold('D', .15)
            print("Pressed D")
            time.sleep(.3)
        
    except Exception as e:
        time.sleep(.6)
        print("Error: ", str(e))
        print("there was an exception")
        print('i: ', i )
        
        #pag.press('s')
    


/home/joe/Downloads/labelsB379.csv
/home/joe/Downloads/testfwdB379.png
/home/joe/Downloads/testdwnB379.png
/home/joe/Dev/robot/auto pilot batch 1/autofwdB2.png
/home/joe/Downloads/testdwnB378.png
scaled_positions:  [0.71484812428278932, -2.0554597620428949, 0.53872038966211444, 0.6925338360519202, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pressed W
/home/joe/Downloads/labelsB1 (1).csv
Error:  -1
there was an exception
i:  0
/home/joe/Downloads/labelsB2 (1).csv
/home/joe/Downloads/testfwdB2.png
/home/joe/Downloads/testdwnB2.png
/home/joe/Dev/robot/auto pilot batch 1/autofwdB2.png
/home/joe/Downloads/testdwnB1.png
scaled_positions:  [0.77942779500596049, -2.587700069414685, 0.54006014831534344, 0.6925338360519202, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pressed W
/home/joe/Downloads/labelsB3 (1).csv
/home/joe/Downloads/testfwdB3.png
/home/joe/Downloads/testdwnB3.png
/home/joe/Dev/robot/auto pilot batch 1/autofwdB2.png
/home/joe/Downloads/testdwnB2.png
scaled_positions:  [0.779847